# Multi-stage workflow for Reddit mood classifier

1. ?????????? ingestion: `python python/data_pipeline.py --output python/data/combined_dataset.parquet` (????? `--skip-pushshift` ?????????????????????????????? timeout).
2. ???????????????????? ??? training ???? GPU: `python python/train_gpu_transformer.py --data-path python/data/combined_dataset.parquet --output-dir model/transformer_distilbert`.
3. ????????????????????????????????????????? timeout ??? training script ??? PyTorch + HuggingFace Transformers ????????? RTX 4070 Ti ???????????????????????????????? (??? `torch.cuda.is_available()`).

?????????????????????????????????????????? ??????? helper ??????????????????????????????????????????? ?.

In [ ]:
import subprocess
import sys
from pathlib import Path

DATA_PATH = Path('python/data/combined_dataset.parquet')
STATS_PATH = Path('python/data/combined_dataset_stats.json')
MODEL_DIR = Path('model/transformer_distilbert')

BASE_DATA_CMD = [
    sys.executable, 'python/data_pipeline.py',
    '--output', str(DATA_PATH),
    '--stats-output', str(STATS_PATH),
]

BASE_TRAIN_CMD = [
    sys.executable, 'python/train_gpu_transformer.py',
    '--data-path', str(DATA_PATH),
    '--output-dir', str(MODEL_DIR),
]

def run_data_pipeline(skip_pushshift: bool = True, max_per_label: int | None = None):
    cmd = BASE_DATA_CMD.copy()
    if skip_pushshift:
        cmd.append('--skip-pushshift')
    if max_per_label:
        cmd += ['--sample-per-label', str(max_per_label)]
    subprocess.run(cmd, check=True)

def run_gpu_training(max_per_label: int | None = None, epochs: float | None = None):
    cmd = BASE_TRAIN_CMD.copy()
    if max_per_label:
        cmd += ['--max-per-label', str(max_per_label)]
    if epochs:
        cmd += ['--epochs', str(epochs)]
    subprocess.run(cmd, check=True)

print(f'Dataset target: {DATA_PATH.resolve()}')
print(f'Model directory: {MODEL_DIR.resolve()}')
print('Call run_data_pipeline() and run_gpu_training() as needed.')
